In [1]:
# Test: Filter Unwanted Files
# Verifies that result, standing, and round files are correctly excluded
# from Draft folders in Google Drive, leaving only player files.
#
# What it does:
#   1. Authenticates with Google Drive and locates latest Season > Pictures
#   2. Scans each Draft folder for files
#   3. Checks that exactly 3 non-player files are excluded per folder
#   4. Reports any mismatches or files that slipped through the filter

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import os
import re

# Configuration
from config import SCOPES, MAIN_FOLDER_ID

print("Starting authentication...")

# Authenticate
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    print("✓ Using existing token.json")
else:
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())
    print("✓ Created new token.json")

service = build('drive', 'v3', credentials=creds)
print("✓ Connected to Google Drive\n")

def get_folders(parent_id, name_pattern=None):
    """Get non-trashed folders from a parent folder."""
    query = f"'{parent_id}' in parents and mimeType='application/vnd.google-apps.folder' and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    folders = results.get('files', [])
    
    if name_pattern:
        folders = [f for f in folders if re.search(name_pattern, f['name'])]
    
    return folders

def get_files(parent_id):
    """Get non-trashed files from a folder, sorted by name."""
    query = f"'{parent_id}' in parents and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    return sorted(files, key=lambda x: x['name'])

def is_player_file(filename):
    """Check if file is a player file (not an overview/backup file)."""
    name_lower = filename.lower()
    
    # Exclude files with these patterns
    if '+' in filename:  # R1+R2, R3+R4
        return False
    if 'result' in name_lower:  # Results, Result, etc.
        return False
    if 'standing' in name_lower:  # Standings, Standing, etc.
        return False
    if re.search(r'^r\d', name_lower):  # Files starting with R followed by digit
        return False
    
    return True

# Find all Season folders first
print("Step 1: Finding all season folders...")
all_season_folders = get_folders(MAIN_FOLDER_ID, r'Season \d+')
print(f"Found {len(all_season_folders)} season folders:")
for folder in all_season_folders:
    print(f"  - {folder['name']}")

# Find Season 5 specifically
print("\nStep 2: Looking for Season 5...")
season_5 = next((f for f in all_season_folders if 'Season 5' in f['name']), None)

if not season_5:
    print("❌ Error: Season 5 not found!")
    print("Available seasons:", [f['name'] for f in all_season_folders])
    exit()

print(f"✓ Found: {season_5['name']}\n")

# Find Pictures folder
print("Step 3: Finding Pictures folder...")
folders_in_season = get_folders(season_5['id'])
print(f"Folders in {season_5['name']}:")
for folder in folders_in_season:
    print(f"  - {folder['name']}")

pictures_folder = next((f for f in folders_in_season if f['name'].lower() == 'pictures'), None)

if not pictures_folder:
    print("❌ Error: Pictures folder not found!")
    exit()

print(f"✓ Found: Pictures folder\n")

# Get ALL Draft folders
print("Step 4: Getting all Draft folders...")
draft_folders = get_folders(pictures_folder['id'], r'\d{8}\s+Draft\s+\d+')

if not draft_folders:
    print("❌ No Draft folders found!")
    print("Let me check what folders exist in Pictures:")
    all_pic_folders = get_folders(pictures_folder['id'])
    for folder in all_pic_folders:
        print(f"  - {folder['name']}")
    exit()

draft_folders = sorted(draft_folders, key=lambda f: int(re.match(r'(\d{8})', f['name']).group(1)))

print(f"✓ Found {len(draft_folders)} Draft folders:")
for folder in draft_folders:
    print(f"  - {folder['name']}")

print("\nSTARTING TESTS\n")

# Test each Draft folder and collect results
issues_found = False
failed_folders = []
detailed_output = []

for draft in draft_folders:
    output = []
    output.append(f"Checking: {draft['name']}")
    
    all_files = get_files(draft['id'])
    
    # Files that should be excluded
    excluded_files = [f for f in all_files if not is_player_file(f['name'])]
    
    # Files that are detected as players
    player_files = [f for f in all_files if is_player_file(f['name'])]
    
    output.append(f"  Total files: {len(all_files)}")
    output.append(f"  Player files detected: {len(player_files)}")
    output.append(f"  Excluded files: {len(excluded_files)}")
    
    folder_has_issues = False
    
    # CHECK: Should have exactly 3 excluded files
    if len(excluded_files) < 3:
        output.append(f"  🚨 ALERT: Expected 3 excluded files, found only {len(excluded_files)}!")
        issues_found = True
        folder_has_issues = True
    elif len(excluded_files) > 3:
        output.append(f"  🚨 ALERT: Expected 3 excluded files, found {len(excluded_files)}!")
        issues_found = True
        folder_has_issues = True
    
    # Show excluded files
    if excluded_files:
        output.append(f"  ✓ Correctly excluded:")
        for f in excluded_files:
            output.append(f"    - {f['name']}")
    else:
        output.append(f"  ⚠️  No files were excluded!")
    
    # Check for potential issues
    for f in player_files:
        name_lower = f['name'].lower()
        if 'round' in name_lower or 'result' in name_lower or 'standing' in name_lower or '+' in f['name']:
            output.append(f"  ⚠️  WARNING: This file should probably be excluded: {f['name']}")
            issues_found = True
            folder_has_issues = True
    
    # Track failed folders
    if folder_has_issues:
        failed_folders.append({
            'name': draft['name'],
            'excluded_count': len(excluded_files),
            'excluded_files': [f['name'] for f in excluded_files]
        })
    
    detailed_output.append('\n'.join(output))

# Print TEST RESULTS first
print("TEST RESULTS\n")

if not issues_found:
    print("✅ All checks passed! No result/round/standing files detected as players.\n")
else:
    print("Issues found:\n")
    for folder in failed_folders:
        print(f"  📁 {folder['name']}")
        print(f"     - Excluded files: {folder['excluded_count']} (expected 3)")
        if folder['excluded_files']:
            print(f"     - Files excluded: {', '.join(folder['excluded_files'])}")
        else:
            print(f"     - Files excluded: None")
        print()

# Then print detailed output
print("DETAILED CHECK RESULTS\n")
for output in detailed_output:
    print(output)
    print()

assert not issues_found, (
    f'{len(failed_folders)} folder(s) with issues:\n' +
    '\n'.join(
        f'  - {f["name"]}: {f["excluded_count"]} excluded (expected 3)'
        for f in failed_folders
    )
)


Starting authentication...
✓ Using existing token.json
✓ Connected to Google Drive

Step 1: Finding all season folders...
Found 5 season folders:
  - Season 5
  - Season 4
  - Season 3
  - Season 2
  - Season 1

Step 2: Looking for Season 5...
✓ Found: Season 5

Step 3: Finding Pictures folder...
Folders in Season 5:
  - Backups
  - Manacore
  - Pictures
✓ Found: Pictures folder

Step 4: Getting all Draft folders...
✓ Found 7 Draft folders:
  - 20250817 Draft 1
  - 20250921 Draft 2
  - 20251026 Draft 3
  - 20251108 Draft 4
  - 20251221 Draft 5
  - 20251228 Draft 6
  - 20260125 Draft 7

STARTING TESTS

TEST RESULTS

✅ All checks passed! No result/round/standing files detected as players.

DETAILED CHECK RESULTS

Checking: 20250817 Draft 1
  Total files: 15
  Player files detected: 12
  Excluded files: 3
  ✓ Correctly excluded:
    - R1+R2.jpeg
    - R3+R4.jpeg
    - Result.jpeg

Checking: 20250921 Draft 2
  Total files: 15
  Player files detected: 12
  Excluded files: 3
  ✓ Correctly ex